In [4]:
import pandas as pd
import numpy as np

data = pd.read_excel('Si_AE.xlsx', sheet_name='ld1')
print(data.head())

          r            3P        3S            2P        2S        1S  \
0  0.000065  8.533000e-08  0.000502  3.660390e-07  0.001746  0.006586   
1  0.000066  8.636000e-08  0.000505  3.704570e-07  0.001757  0.006625   
2  0.000066  8.740300e-08  0.000508  3.749280e-07  0.001767  0.006665   
3  0.000066  8.845700e-08  0.000511  3.794530e-07  0.001778  0.006705   
4  0.000067  8.952500e-08  0.000514  3.840330e-07  0.001788  0.006746   

            3P2           3S2           2P3           2S4           1S5  
0  5.557921e-12  3.269029e-08  2.384174e-11  1.137316e-07  4.289682e-07  
1  5.658861e-12  3.308475e-08  2.427472e-11  1.151039e-07  4.341444e-07  
2  5.761671e-12  3.348398e-08  2.471553e-11  1.164929e-07  4.393831e-07  
3  5.866244e-12  3.388802e-08  2.516436e-11  1.178986e-07  4.446850e-07  
4  5.972800e-12  3.429693e-08  2.562136e-11  1.193212e-07  4.500508e-07  


In [5]:
def cutoff_radius(r_cl):
    closest_index, closest_number = min(enumerate(data['r']),key=lambda x: abs(x[1]-r_cl))
    return(closest_index)

In [6]:
print(data.iloc[1,1])

8.636e-08


In [7]:
def norm_AE(r_cl,orbital_label):
    norm_AE = 0
    for i in range(0,cutoff_radius(r_cl)):
        norm_AE = norm_AE + ((data.iloc[i,orbital_label])**2 + (data.iloc[i+1,orbital_label])**2)*((data.iloc[i+1,0])-data.iloc[i,0])/2
    return(norm_AE)    

In [8]:
print(norm_AE(1,6))

0.008307433076215103


In [9]:
def first_derivative(r_cl,orbital_label):
    index = cutoff_radius(r_cl)
    return((data.iloc[index-2,orbital_label]-8*data.iloc[index-1,orbital_label]+8*data.iloc[index+1,orbital_label]-data.iloc[index+2,orbital_label])/(3*(data.iloc[index+2,0]-data.iloc[index-2,0])))


In [10]:
print(first_derivative(1,6))

-1.0164516924765794


In [11]:
def second_derivative(r_cl,orbital_label):
    index = cutoff_radius(r_cl)
    r_cl_02 = data.iloc[index-2,0]
    r_cl_01 = data.iloc[index-1,0]
    r_cl_1 = data.iloc[index+1,0]
    r_cl_2 = data.iloc[index+2,0]
    return((first_derivative(r_cl_02,orbital_label)-8*first_derivative(r_cl_01,orbital_label)+8*first_derivative(r_cl_1,orbital_label)-first_derivative(r_cl_2,orbital_label))/(3*(data.iloc[index+2,0]-data.iloc[index-2,0])))


In [12]:
print(second_derivative(1,6))

-0.7129776040404567


In [13]:
def matrix_gen_A(r_cl):
    r = data.iloc[cutoff_radius(r_cl),0]
    print(r)
    A = np.zeros((4,4))
    A[0,0]=1; A[0,1]=r**2; A[0,2]=r**3; A[0,3]=r**4
    A[1,1]=2*r; A[1,2]=3*r**2; A[1,3]=4*r**3
    A[2,1]=2; A[2,2]=6*r; A[2,3]=12*r**2
    return(A)
    

In [14]:
print(matrix_gen_A(1))

1.002947004289
[[ 1.          1.00590269  1.00886709  1.01184023]
 [ 0.          2.00589401  3.01770808  4.03546837]
 [ 0.          2.          6.01768203 12.07083232]
 [ 0.          0.          0.          0.        ]]


In [15]:
def angular_momentum(orbital_label):
    if orbital_label == 6 or 8:
        l = 1
    elif orbital_label == 7 or 9 or 10:
        l = 0
    else:
        raise ValueError
    return(l)

In [19]:
def vector_gen_b(r_cl,orbital_label):
    r = data.iloc[cutoff_radius(r_cl),0]
    l = angular_momentum(orbital_label)
    b = np.zeros((4,1))
    b[0]=np.log(data.iloc[cutoff_radius(r),orbital_label]/(r**(l+1)))
    b[1]=first_derivative(r,orbital_label)/data.iloc[cutoff_radius(r),orbital_label]-(l+1)/r
    b[2]=(second_derivative(r,orbital_label)*data.iloc[cutoff_radius(r),orbital_label]-first_derivative(r,orbital_label)**2)/(data.iloc[cutoff_radius(r),orbital_label])**2+(l+1)/r**2
    print(l)
    return(b)

In [21]:
print(vector_gen_b(1,7))

1
[[ -1.0128219 ]
 [  2.03792655]
 [-13.05708993]
 [  0.        ]]


In [23]:
def Gauss_Jordan_elimination(A,b):
    A = np.hstack((A, b.reshape(-1, 1)))
    rows, cols = A.shape
    for i in range(rows):
        # 找主元列
        for j in range(i, cols - 1):
            if A[i, j] != 0:
                # 交换行，保证主元在对角线位置
                if i != j:
                    A[[i, j]] = A[[j, i]]
                # 归一化主元
                A[i] /= A[i, j]
                # 消去主元所在列的其他行
                for k in range(rows):
                    if k != i:
                        A[k] -= A[k, j] * A[i]
                break
    return(A)